<a href="https://colab.research.google.com/github/redrum88/ebay_scraping/blob/main/ebay_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Product information

## Import libraries

In [ ]:
import os
import time
import requests
import urllib.parse
import pandas as pd
from PIL import Image
from requests.exceptions import MissingSchema


## Function to get product information

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Function to create new directory if not exist
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
# url = "https://www.ebay.co.uk/itm/314500756678?epid=27023784973&hash=item4939b4b4c6:g:TK8AAOSwYJVkKyZR"

def get_product_info(url):

  headers = {
      'Accept-Language:': 'lt-LT,lt;q=0.9,en-US;q=0.8,en;q=0.7,ru;q=0.6,pl;q=0.5',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
  }

  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Get product title
  title = soup.find('h1', 'x-item-title__mainTitle').get_text()

  # Get all images
  image_urls = [] # Empty list to store images

  # Find all images and store in image_urls empty list
  for div in soup.find_all('div', {'class': 'ux-image-carousel-item'}):
      img = div.find('img')
      if img is not None:
          if 'src' in img.attrs:
              image_urls.append(img['src'])
          elif 'data-src' in img.attrs:
              image_urls.append(img['data-src'])

  # Find product price
  price = soup.find('div', class_='x-price-primary').text.strip()

  # Find all specifics labels and values
  labels = []
  values = []

  specifics_box = soup.find('div', class_='vim x-about-this-item')
  # Labels
  all_labels = specifics_box.find_all('div', class_='ux-labels-values__labels')
  for label in all_labels:
    labels.append(label.text)
  # Values
  all_values = specifics_box.find_all('div', class_='ux-labels-values__values')
  for value in all_values:
    values.append(value.text)
  # Add data to dictionary
  specifics_dict = {label: value for label, value in zip(labels, values)}
  specifics_dict['Title'] = title
  specifics_dict['Price'] = price
  specifics_dict['Image URLs'] = image_urls

  # Download images
  # Create directory structure
  directory = os.path.join('images', title.replace(' ', '_'))
  create_directory(directory)
  downloaded_filenames = []
  downloaded_directory = []

  # Loop each image URL in the row
  total_images = 0  # Initialize counter
  counter = 0

  for url in image_urls:
        
    # if not url:
    #     continue
            
    # Get filename from URL
    filename = os.path.basename(urllib.parse.urlsplit(url).path)
        
    # # If file exists then skip it
    # if os.path.exists(os.path.join(directory, filename)):
    #     downloaded_filenames.append(filename)
    #     continue

    try:
        # Download image
        #print(url)
        response = requests.get(url)
        with open(os.path.join(directory, str(counter)+'_'+filename), 'wb') as f:
            f.write(response.content)
        image_path = os.path.join(directory, str(counter)+'_'+filename)
        #resize_image(image_path, 100)  # Resize image by percentage. 50 - half size
        downloaded_filenames.append(str(counter)+'_'+filename)
        total_images += 1 # Icrement total images counter
        counter += 1  # Increment counter 
        time.sleep(1)
    except AttributeError:
        print("Error: 'float' object has no attribute 'split'")
    except MissingSchema:
        print(f"Error: Invalid URL {url!r}: No scheme supplied. Perhaps you meant https://{url}?")


  specifics_dict['downloaded_filenames'] = downloaded_filenames
  specifics_dict['path_dir'] = directory
  specifics_dict['total_images'] = total_images
  return specifics_dict

## Check if function works

In [ ]:
test = get_product_info("https://www.ebay.co.uk/itm/314500756678?epid=27023784973&hash=item4939b4b4c6:g:TK8AAOSwYJVkKyZR")

https://i.ebayimg.com/images/g/TK8AAOSwYJVkKyZR/s-l500.jpg
https://i.ebayimg.com/images/g/UCYAAOSw51NkKyZR/s-l500.jpg
https://i.ebayimg.com/images/g/ShQAAOSw7o1kKyZR/s-l500.jpg
https://i.ebayimg.com/images/g/6dYAAOSwCS9kKyZQ/s-l500.jpg
https://i.ebayimg.com/images/g/sMsAAOSw2ApkKyZR/s-l500.jpg
https://i.ebayimg.com/images/g/OMAAAOSw1V9kKyZQ/s-l500.jpg
https://i.ebayimg.com/images/g/uXkAAOSwVT5kKyZR/s-l500.jpg
https://i.ebayimg.com/images/g/dAUAAOSwFYNkKyZR/s-l500.jpg
https://i.ebayimg.com/images/g/QTcAAOSw6-BkKyZQ/s-l500.jpg
https://i.ebayimg.com/images/g/7lQAAOSwtilkKyZQ/s-l500.jpg
https://i.ebayimg.com/images/g/sxsAAOSwrddkKyZR/s-l500.jpg
https://i.ebayimg.com/images/g/FgUAAOSwwKJkKyZR/s-l500.jpg


In [ ]:
url_df = pd.read_csv('/content/ebay_urls.csv')
url_df.head()

,Title,Title_URL,Image,Price
0,NISSAN QASHQAI DOOR REAR LEFT SIDE SILVER KY0 ...,https://www.ebay.co.uk/itm/404233779650?hash=i...,https://i.ebayimg.com/thumbs/images/g/bfAAAOSw...,£399.00
1,NISSAN X TRAIL ELECTRIC HAND PARK BRAKE SWITCH...,https://www.ebay.co.uk/itm/404231133777?hash=i...,https://i.ebayimg.com/thumbs/images/g/lkoAAOSw...,£34.99
2,BMW X5 HEADREST DISPLAY SCREEN VEBA FRONT RIGH...,https://www.ebay.co.uk/itm/354684267247?hash=i...,https://i.ebayimg.com/thumbs/images/g/LJEAAOSw...,£89.99
3,VOLKSWAGEN PASSAT ABS PUMP MODULATOR 2.0 DIESE...,https://www.ebay.co.uk/itm/314500761210?hash=i...,https://i.ebayimg.com/thumbs/images/g/y-UAAOSw...,£25.00
4,TOYOTA AYGO BUMPER FRONT POLAR WHITE 068 52119...,https://www.ebay.co.uk/itm/354699721257?hash=i...,https://i.ebayimg.com/thumbs/images/g/JVsAAOSw...,£399.00


## Write loop to download images and product specifics for all product urls from .csv

In [ ]:
import random
import os

url_df = pd.read_csv('/content/ebay_urls.csv')
all_products = []

def check_folder_size(folder_path):
  num_files = len(os.listdir(folder_path))
  if num_files >= 9:
    print("There are already 9 files in the folder, skipping.")
    return False
  else:
    return True

all_products = []

for index, row in url_df.iterrows():
  print(f"Processing row {index} of {len(url_df)}")
  url = row['Title_URL']
  title = row['Title']
  
  if url is None or title is None:
    print(f"Skipping row {index}: URL or title missing.")
    continue
    
  if not check_folder_size(os.getcwd()):
    continue
  
  try:
    start_time = time.time()
    request = get_product_info(url)
    all_products.append(request)
    end_time = time.time()  # end timer
    download_time = end_time - start_time
    print(f'In {download_time:.2f} sec copied specifics for {title}')
  except AttributeError:
    print(f"No title or URL found for {title}")
    continue
  
  time.sleep(random.randint(1, 3))

print(len(all_products))








Processing row 0 of 7391
In 14.40 sec copied specifics for NISSAN QASHQAI DOOR REAR LEFT SIDE SILVER KY0 J11 2014-2021 H2101HV0MA
Processing row 1 of 7391
In 14.30 sec copied specifics for NISSAN X TRAIL ELECTRIC HAND PARK BRAKE SWITCH & TRIM 251756FR0A MK3 2014 - 2021
Processing row 2 of 7391
In 16.13 sec copied specifics for BMW X5 HEADREST DISPLAY SCREEN VEBA FRONT RIGHT LEFT PAIR E53 1999 - 2007
Processing row 3 of 7391
In 14.90 sec copied specifics for VOLKSWAGEN PASSAT ABS PUMP MODULATOR 2.0 DIESEL 3AA614109AQ B7 2010 - 2016
Processing row 4 of 7391
In 15.57 sec copied specifics for TOYOTA AYGO BUMPER FRONT POLAR WHITE 068 521190H918 MK2 2014 - 2018
Processing row 5 of 7391
In 15.84 sec copied specifics for HONDA CIVIC CAMERA REVERSING REAR VIEW 39530TV0E01 MK9 2011 - 2017
Processing row 6 of 7391
In 14.36 sec copied specifics for MERCEDES A CLASS CAMERA FRONT W177 2018 - 2023 A2479002107
Processing row 7 of 7391
In 14.00 sec copied specifics for AUDI A3 GATEWAY CONTROL MODULE EC

KeyboardInterrupt: ignored

In [ ]:
products = pd.DataFrame(all_products)
products.columns

Index(['Condition', 'Seller notes', 'Brand', 'Make', 'Placement on Vehicle',
       'Type', 'Mileage', 'Vehicle Identification Number (VIN)', 'Grade',
       'Paint Code', 'Year', 'Stock Number', 'Model', 'Tag Number',
       'Manufacturer Part Number', 'Reference OE/OEM Number',
       'ExactLinc Number', 'Item Name', 'Number of Pieces', 'Title', 'Price',
       'Image URLs', 'downloaded_filenames', 'path_dir', 'total_images',
       'Condition and Options', 'Superseded Part Number', 'UPC', 'EAN',
       'Tyre Size', 'Tyre Condition', 'Aspect Ratio', 'Rim Condition',
       'Offset', 'PCD', 'Bore Size (mm)', 'Rim Width', 'Package Quantity',
       'Wheel Diameter', 'Tyre Width', 'Wheel Width', 'Colour',
       'Other Part Number', 'Other Part Number 2', 'MPN'],
      dtype='object')